In [1]:
import numpy as np
import pandas as pd
import sys, re
import shutil

导入数据，format： 
id    model_score   question   answer   label

In [3]:
## 每个问句都是返回前三sore的问答句pair，label暂时都为0，方便之后处理
pada1 = pd.read_csv('test1.txt', sep='\t')
pada2 = pd.read_csv('test2.txt', sep='\t')
pada3 = pd.read_csv('test3.txt', sep='\t')
dicts = [pada1, pada2, pada3]
##raters = pd.read_csv('raters.txt', sep='\t')

dicts = []  ### dicts[pada0, pada1, pada2, pada3.......padak]

In [188]:
def labeled(dicts, raters):
    tmp = dicts
    rater = raters.values.tolist()
    if tmp == None:
        print("Please load your test set!")
        return 0
    lens = 0
    if len(tmp) != len(raters):
        lens = len(tmp)
    else:
        lens = len(raters)
    
    for i in range(lens):
        for j in range(len(tmp[i]['answer'])):
            if tmp[i]['answer'][j] == rater[i][1]:
                tmp[i]['label'][j] = 1
                continue
            elif tmp[i]['answer'][j] == rater[i][2]:
                tmp[i]['label'][j] = 1
                continue
            elif tmp[i]['answer'][j] == rater[i][3]:
                tmp[i]['label'][j] = 1
                continue  
            else:
                tmp[i]['label'][j] = 0
    return tmp, rater

In [185]:
values, ss = labeled(dicts, raters)

/home/victor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/victor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/victor/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [187]:
values

[   id  score     question                                             answer  \
 0  29   30.2    Hello，晚上好  晚上好，不管白天还是黑夜都竭诚为你服务的人工智能机器人达尔文。我对答如流，无所不知，不知疲倦...   
 1  31   23.2  嗨喽 你知道我是谁吗？                               我想你是我的朋友，也会成为我的忠实粉丝。   
 2  33   20.1  几天不见，变聪明了一点  我可是划时代的人工智能机器人，我可是不知疲倦的在不断的学习中，学习是让人变聪明的最直接的方式...   
 
    label  
 0      1  
 1      0  
 2      0  ,
    id  score     question                                             answer  \
 0  29   30.2    Hello，晚上好  晚上好，不管白天还是黑夜都竭诚为你服务的人工智能机器人达尔文。我对答如流，无所不知，不知疲倦...   
 1  31   23.2  嗨喽 你知道我是谁吗？                               我想你是我的朋友，也会成为我的忠实粉丝。   
 2  33   20.1  几天不见，变聪明了一点  我可是划时代的人工智能机器人，我可是不知疲倦的在不断的学习中，学习是让人变聪明的最直接的方式...   
 
    label  
 0      0  
 1      1  
 2      0  ,
    id  score     question                                             answer  \
 0  29   30.2    Hello，晚上好  晚上好，不管白天还是黑夜都竭诚为你服务的人工智能机器人达尔文。我对答如流，无所不知，不知疲倦...   
 1  31   23.2  嗨喽 你知道我是谁吗？                               我想你是我的朋友，也会成为我的忠实粉丝。   
 2  33 

In [8]:
for q in dicts:
    print(q["question"][1])

嗨喽 你知道我是谁吗？
嗨喽 你知道我是谁吗？
嗨喽 你知道我是谁吗？


'我想你是我的朋友，也会成为我的忠实粉丝'

calculate the matrix
df.values.tolist()

In [155]:
sum = 0
### precision@1
for i in range(len(dicts)):
    sum += dicts[i]["label"][0]
    
precision1 = sum / len(dicts)



In [8]:
def precision(dicts, k):
    if dicts == None:
        print("Please load your test set!")
        return 0
    if k <= 0 or k > len(dicts):
        print("please input a valid value for k, which is from 1  to" + len(dicts))
        return 0
    
    sum = 0
    ## 对于sample中每一个rank为k的问答句pair precision@k = sum / sample中所有rank为k的问句总数
    for q in dicts:
        sum += q["label"][k-1]
    res = sum / len(dicts)
    return res



## MAP value
def mAP(dicts, k):
    
    if dicts == None:
        print("Please load your test set!")
        return 0
    if k <= 0 or k > len(dicts):
        print("please input a valid value for k, which is from 1  to" + len(dicts))
        return 0
    
    sum1 = 0
    for q in dicts:
        sum2 = 0
        Nq = 0
        for i in range(k):
            sum2 += precision(dicts, i+1) * (precision(dicts, i+1) * len(dicts))
        for j in range(len(q)):
            Nq += q["label"][j]
            
        sum1 += sum2 / Nq
    return sum1 / len(dicts)

## rank(i) ???
def rank(q):
    return 1

## MRR
def mRR(dicts):
    if dicts == None:
        print("Please load your test set!")
        return 0
    
    sum1 = 0
    for q in dicts:
        sum1 += 1 / rank(q)
    return sum1 / len(dicts)


## Z is a constant number
Z = 2.331

## nDCG
def nDCG(dicts, k):
    if dicts == None:
        print("Please load your test set!")
        return 0
    if k <= 0 or k > len(dicts):
        print("please input a valid value for k, which is from 1  to" + len(dicts))
        return 0
    
    sum1 = 0
    for q in dicts:
        sum2 = 0
        for i in range(k):
            sum2 += (2**(precision(dicts, i+1) * len(dicts)) - 1) / np.log(1 + (i+1))
        
        sum1 += sum2 / Z
                     
        return sum1 / len(dicts)

In [9]:
nDCG(dicts, 1)

0.20630559715271893